# Import Packages

In [83]:
# Base packages
import numpy as np
import pandas as pd
from matplotlib.pyplot import get_cmap
from matplotlib.colors import rgb2hex

# Holoviews packages
import holoviews as hv
import hvplot.pandas
from bokeh.io import output_notebook
from bokeh.palettes import varying_alpha_palette, Category20
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.models import HoverTool, ColumnDataSource, CDSView, GroupFilter, NumeralTickFormatter, LinearColorMapper, ColorBar, TabPanel, Tabs
import panel as pn
output_notebook()
hv.extension('bokeh')
pn.extension(throttled=True)

# Custom post processing functions
import TEVA_Post_Processing as post

Loading BokehJS ...

# Import TEVA Output Files

In [84]:
# Import CC and DNF output files
# ccs = pd.read_excel('ccs_2DOC_CAMELS.xlsx', sheet_name='CCEA_Low')
# dnfs = pd.read_excel('dnfs_2DOC_CAMELS.xlsx', sheet_name='DNFEA_Low')

ccs = pd.read_excel('Sample_Data/ccs_2DOC_CAMELS_1_S_True_60_60_TEVA007.xlsx', sheet_name='CCEA_Low')
dnfs = pd.read_excel('Sample_Data/dnfs_2DOC_CAMELS_1_S_True_60_60_TEVA007.xlsx', sheet_name='DNFEA_Low')

# Import observation data
data = pd.read_csv('Sample_Data/test_observations.csv')

# Run Post-Processing Functions
These functions, imported from the "TEVA_Post_Processing.py" file, help parse the Excel spreadsheets into a more usable form.

In [85]:
# List of the CCs composing each DNF
all_ccs = post.parse_dnf(dnfs)
all_ccs_flat = post.flatten(all_ccs)

# List of the features composing each CC
cc_features = post.parse_cc(ccs)
unique_features = pd.unique(post.flatten(cc_features))
all_features_flat = post.flatten(cc_features)

# List of the unique CCs across all DNFs
unique_ccs = (np.unique(all_ccs_flat))

# Fitness contours
x_fit, y_fit, z_fit, fitness = post.fitness_contours(1000, dnfs, ccs)

# CC feature usage image matrix
cc_matrix = post.CC_feature_heatmap(unique_features, cc_features)

# CC and DNF lengths (needed later for plotting)
cc_len = np.arange(1, max(ccs['order']) + 1, 1)
dnf_len = np.arange(1, max(dnfs['order']) + 1, 1)

# Stacked feature and cc dataframes for bar chart
stacked_features, stacked_feature_names = post.stacked_features(ccs, unique_features, cc_features, all_features_flat)
stacked_ccs, stacked_cc_names = post.stacked_ccs(dnfs, unique_ccs, all_ccs, all_ccs_flat)

# feature value ranges by cc
feature_values = post.feature_ranges_by_feature(ccs)

In [ ]:
# TEST CELL
# hexbin setup

# hex_feat = 16

# a = np.array(feature_values[hex_feat])

# p = figure(match_aspect=True,
#            x_axis_label='Feature Minimum',
#            y_axis_label='Feature Maximum',
#         #    title=unique_features[hex_feat]
#            )

# r, bins = p.hexbin(x=a[:,0], y=a[:,1],
#                    size=0.04,
#                    aspect_scale=1)

# p.scatter(a[:,0], a[:,1], color='white', line_color='black', size=10)

# p.add_tools(HoverTool(
#     tooltips=[('count', '@c')],
#     mode='mouse', point_policy='follow_mouse', renderers=[r]
# ))

# show(p)


# Set up the Dashboard
    > Custom color maps for cc/dnf plot
    

In [87]:
# Custom color maps
'''
Create custom colormaps, one for CCs, one from DNFs, and one for fitness contours.
Colormaps can range from 0 to 256, but it is best to trim the lightest and darkest portions out.
'''
cc_colors = []
dnf_colors = []

# CCs and DNFs
for i in range(20,220):
    cc_colors.append(rgb2hex(get_cmap('Blues_r')(i)))
    dnf_colors.append(rgb2hex(get_cmap('Oranges_r')(i)))

# Fitness contours
contour_colors = varying_alpha_palette(color='black', start_alpha=150, end_alpha=10)

# For CC feature heatmap
cc_heatmap_colormap = []
for i in range(0,256):
    cc_heatmap_colormap.append(rgb2hex(get_cmap('Blues')(i)))

In [88]:
# Bokeh Data Sources
'''
Bokeh uses a data structure called a "Column Data Source" (CDS) for plotting.
The easiest way to create them with your data is by passing your data as a disctionary.
'''

# column data source for CCs
cc_plot_data = {'x_values': ccs['cov'],
                'y_values': ccs['ppv'],
                'CC': ccs['Unnamed: 0'],
                'Order': ccs['order'],
                'Features': cc_features}
cc_plot_source = ColumnDataSource(data=cc_plot_data)

# column data source for DNFs
dnf_plot_data = {'x_values': dnfs['cov'],
                 'y_values': dnfs['ppv'],
                 'Order': dnfs['order'],
                 'DNF': dnfs['Unnamed: 0'],
                 'CCs': all_ccs}
dnf_plot_source = ColumnDataSource(data=dnf_plot_data)

# column data source for fitness contours
dnf_cont_data = {'x_values': x_fit,
                 'y_values': y_fit,
                 'z_values': z_fit}
dnf_cont_source = ColumnDataSource(dnf_cont_data)

# CC feature image
cc_image_data = {
    'image': [cc_matrix],
    'x': [0],
    'y': [0],
    'dw': [len(unique_features)],
    'dh': [len(unique_features)]
}

cc_image_data = ColumnDataSource(data=cc_image_data)

In [89]:
# Bokeh Figure Setup
# tooltips to display when you hover over a data point
h = 600
w = 800

dnf_TOOLS = [
    ('DNF #', '@DNF'),
    ('Order', '@Order'),
    ('PPV', '@y_values'),
    ('COV', '@x_values'),
    ('CCs', '@CCs')]

cc_TOOLS = [
    ('CC #', '@CC'),
    ('Order', '@Order'),
    ('PPV', '@y_values'),
    ('COV', '@x_values'),
    ('Features', '@Features')]

#### Figure 1
p = figure(width = w, height = h,
           y_range=(0,1.05),
           x_range=(0,1.05),
           x_axis_label='Observation Coverage',
           y_axis_label='Positive Predictive Value',
           hidpi=True,
           tools='crosshair, pan, tap, wheel_zoom, zoom_in, zoom_out, box_zoom, undo, redo, reset, save, lasso_select, help')

# cont_levels = np.linspace(min(fitness), max(fitness), 10)
cont_levels = np.linspace(min(fitness), max(fitness), 10)
contour_renderer = p.contour(x_fit, y_fit, z_fit,
                             levels=cont_levels,
                             line_color='gray',
                             fill_color=contour_colors,
                             line_dash='dashed')
# Plot CCs, colored by order
for i in range(0, len(cc_len)):
    cc_plots = p.scatter('x_values', 'y_values', source=cc_plot_source,
              view=CDSView(filter=GroupFilter(column_name='Order', group=len(cc_len) - i)),
              size=12,
              marker='square',
              line_color='white',
              fill_color=linear_cmap('Order', cc_colors, low=min(ccs['order']), high=max(ccs['order'])),
              hover_color='black',
              legend_label='CC Order {}'.format(len(cc_len) - i),
              fill_alpha=1)

# Add hover tool for CCs
p.add_tools(HoverTool(tooltips=cc_TOOLS,
                      mode='mouse',
                      point_policy='follow_mouse'))

# Plot DNFs, colored by order
all_dnf_plots = []
for i in range(0, len(dnf_len)):
    dnf_plot = p.scatter('x_values', 'y_values', source=dnf_plot_source,
              view=CDSView(filter=GroupFilter(column_name='Order', group=len(dnf_len) - i)),
              size=13,
              marker='circle',
              line_color='white',
              fill_color=linear_cmap('Order', dnf_colors, low=min(dnfs['order']), high=max(dnfs['order'])),
              hover_color='black',
              legend_label='DNF Order {}'.format(len(dnf_len) - i),
              fill_alpha=1)
    all_dnf_plots.append(dnf_plot)

# Add seperate hover tool for DNFs
p.add_tools(HoverTool(renderers = all_dnf_plots,
                      tooltips=dnf_TOOLS,
                      mode='mouse',
                      point_policy='follow_mouse'))

# Add color bar for fitness contours
colorbar = contour_renderer.construct_color_bar(height=int(h/2),
                                                location=(0,int(h/4)),
                                                formatter = NumeralTickFormatter(format='0 a'),
                                                bar_line_color='black',
                                                major_tick_line_color='black',
                                                title='Fitness')

# General formatting
p.legend.click_policy='hide'
p.legend.location='bottom_left'
p.add_layout(colorbar, 'right')
nonselection_fill_alpha=0.2

In [90]:
# Bokeh cc feature heatmap setup
# Hover tooltip
cc_image_hover = [
    ('Count', '@image')
]

# Figure setup
cc_heatmap = figure(height=400,
                    aspect_ratio=1,
                    x_range=unique_features,
                    y_range=unique_features,    
                    match_aspect=True,
                    tools=['hover', 'crosshair', 'save'],
                    tooltips=cc_image_hover)

# Color map for data
img_color_mapper = LinearColorMapper(palette=cc_heatmap_colormap, low=0, high=len(unique_features))
# Imshow
img = cc_heatmap.image(source=cc_image_data, x='x', y='y', dw='dw', dh='dh', color_mapper=img_color_mapper)
# Create colorbar
color_bar = ColorBar(color_mapper=img_color_mapper, label_standoff=12, major_tick_line_color='black')
# Add colorbar to figure
cc_heatmap.add_layout(color_bar, 'right')

# color for nan values
img.glyph.color_mapper.nan_color = (0, 0, 0, 0)

# rotate x labels
cc_heatmap.xaxis.major_label_orientation = np.pi/2

# turn off grid
cc_heatmap.xgrid.visible = False
cc_heatmap.ygrid.visible = False

# Add to tab
tab1 = TabPanel(child=cc_heatmap, title='Feature Pairing')

In [91]:
#### FIGURE 2
p2 = figure(width = w, height = int(h * 0.8),
            x_range=stacked_features['Feature'],
            x_axis_label='Feature',
            y_axis_label='Count',
            hidpi=True,
            tools='crosshair, reset, save, help')

p2.vbar_stack(stacked_feature_names,
              x='Feature',
              width=0.6,
              color=Category20[len(cc_len)],
              source=stacked_features,
              legend_label=stacked_feature_names)

# General formatting
p2.xaxis.major_label_orientation = 1
p2.y_range.start = 0
p2.legend.location = 'top_right'
p2.legend.orientation = 'vertical'

tab2 = TabPanel(child=p2, title='CC: Feature Usage')

#### FIGURE 3

p3 = figure(width = w, height = int(h * 0.8),
            x_range=stacked_ccs['CC'],
            x_axis_label='CC',
            y_axis_label='Count',
            hidpi=True,
            tools='crosshair, reset, save, help')

p3.vbar_stack(stacked_cc_names,
              x='CC',
              width=0.6,
              color=Category20[len(dnf_len)],
              source=stacked_ccs,
              legend_label=stacked_cc_names)

# General formatting
p3.xaxis.major_label_orientation = 1
p3.y_range.start = 0
p3.legend.location = 'top_right'
p3.legend.orientation = 'vertical'

tab3 = TabPanel(child=p3, title='DNF: CC Usage')

In [92]:
# Function for updating plots in the CC explorer
def feature_plotter(selected_cc):
    fig = []
    for i in range(len(cc_features[selected_cc])):
        fig.append(data[cc_features[selected_cc][i]].dropna().hvplot.kde(height=200, width=300, hover=False))
        fig[i].opts(shared_axes=False, toolbar=None)

        # add feature ranges here?
        # where = (x_plot > ast.literal_eval(ccs[var[i]][cc])[0]) & (x_plot < ast.literal_eval(ccs[var[i]][cc])[1]),


    return pn.FlexBox(objects=fig)

# Widget for selecting CC to plot
dropdown_options = list(np.sort(unique_ccs.astype(int)))
cc_select = pn.widgets.Select(options=dropdown_options, width=75, name='CC Select', description='Select a CC to view features.')

# Bind function to widget
dynamic_subplots = pn.bind(feature_plotter, cc_select)

In [93]:
# app = pn.Row(pn.Column(p, cc_heatmap), cc_select, dynamic_subplots)
app = pn.Row(pn.Column(p, Tabs(tabs=[tab1, tab2, tab3])), cc_select, dynamic_subplots)

print(app)

Row
    [0] Column
        [0] Bokeh(figure)
        [1] Bokeh(Tabs)
    [1] Select(description='Select a CC t..., name='CC Select', options=[np.int64(0), ...], value=np.int64(0), width=75)
    [2] ParamFunction(function, _pane=FlexBox, defer_load=False)


In [94]:
# Run in browser
app.show()

# Run in notebook
# For some reason interactivity in notebook is not working
# app

Launching server at http://localhost:61319


In [95]:
# a = data[cc_features[20][0]].dropna().hvplot.kde(height=200, width=300, hover=False)
# b = p.circle([0], [0], size=20)
# ccs.iloc[0].iloc[14:]